In [ ]:
import pandas as pd
import numpy as np
import datetime
import math
Billing = pd.read_csv("FullData/Billing.csv")
DiseaseCase = pd.read_csv("FullData/DiseaseCase.csv")

In [ ]:
# Quick looks
print("Billing information -----")
print(Billing.isnull().sum())
print(len(Billing['Patient_ID'].unique()))
print(len(Billing.index))
print("\nDisease information -----")
print(len(DiseaseCase.index), len(DiseaseCase['Patient_ID'].unique()))
print(DiseaseCase['Disease'].value_counts())

In [ ]:
# Remove rows w/o service date
Billing = Billing.dropna(subset=['ServiceDate'])
print(len(Billing.index))

In [ ]:
# Prepare disease case data
DiseaseCase['Disease'] = DiseaseCase['Disease'].replace("Diabetes Mellitus (ml)","Diabetes Mellitus") # Make all DM2 diagnosis' the same
DiseaseCase['DateOfOnset'] = pd.to_datetime(DiseaseCase.DateOfOnset,format='%Y-%m-%d')
DiseaseCase = DiseaseCase.sort_values(by='DateOfOnset', ascending = True)
DiseaseCase = DiseaseCase.drop_duplicates(subset = ['Patient_ID'])
print(len(DiseaseCase['Patient_ID'].unique()), len(DiseaseCase.index))

In [ ]:
# Formatting 
DM2DF = Billing[pd.to_numeric(Billing.DiagnosisCode_calc,errors='coerce').notnull()] # convert diagnosis codes to floats
DM2DF['DiagnosisCode_calc'] = pd.to_numeric(DM2DF['DiagnosisCode_calc']) # convert diagnosis codes to floats
DM2DF['ServiceDate'] = pd.to_datetime(DM2DF.ServiceDate,format='%Y-%m-%d')
DM2DF = pd.merge(DM2DF,DiseaseCase[['Patient_ID','DateOfOnset']], on='Patient_ID')

### Get DM2 patients with a complication

In [ ]:
## The 10 Complications (can be improved)
complications = {
    'AP' : [float(item/100) for item in range(41300,41400)], # http://www.icd9data.com/2013/Volume1/390-459/410-414/413/default.htm
    'AS' : [float(item/100) for item in range(44000,44100)], # http://www.icd9data.com/2012/Volume1/390-459/440-449/440/default.htm
    'ICHD' : [float(item/100) for item in range(41400,41500)], # http://www.icd9data.com/2015/Volume1/390-459/410-414/414/default.htm
    'DD' : [311], # http://www.icd9data.com/2012/Volume1/290-319/300-316/311/default.htm
    'DNP' : [float(item/100) for item in range(58500,58600)] + [581.81], # include 250.4? http://www.icd9data.com/2012/Volume1/580-629/580-589/585/default.htm
    'DNU' : [float(item/100) for item in range(35700,35800)], # include 250.6? 357.2 is the exact code http://www.icd9data.com/2012/Volume1/320-389/350-359/357/default.htm 
    'DR' : [float(item/100) for item in range(36200,36210)], # http://www.icd9data.com/2015/Volume1/320-389/360-379/362/default.htm
    'HL' : [389.9], #http://www.icd9data.com/2012/Volume1/320-389/380-389/389/default.htm
    'MI' : [float(item/100) for item in range(41000,41100)], #http://www.icd9data.com/2014/Volume1/390-459/410-414/410/default.htm
    'PVD' : [float(item/100) for item in range(43300,43400)] #http://www.icd9data.com/2013/Volume1/390-459/440-449/443/default.htm
}

In [ ]:
# Print # of patients per complication
for k,v in complications.items() :
    compRows = DM2DF.loc[DM2DF['DiagnosisCode_calc'].isin(v)]
    uniqPatients = compRows['Patient_ID'].unique()
    print(k, "has", len(uniqPatients), "unique patients")

In [ ]:
# Find period between DM2 diagnosis and 1st complication diagnosis
def DiseasePeriod(compDF) :
    compIDs = compDF.sort_values(by='ServiceDate', ascending = True)
    compIDs = compIDs[compIDs['ServiceDate'] > compIDs['DateOfOnset']]
    compIDs = compIDs.drop_duplicates(subset = ['Patient_ID'])
    compIDs = compIDs[['Patient_ID', 'ServiceDate']]
    compIDs.columns = ['Patient_ID', 'CompOnset']
    return compIDs

In [ ]:
# Create negative sample using random sampling (can be improved)
from random import sample
def negativeSamples(posDF) :
    # Negative set
    NegBillingSet = DM2DF.loc[~DM2DF['Patient_ID'].isin(posDF['Patient_ID'])]
    
    # Remove patients with only 1 visit and more than 50 visits
    NegBillingSet = NegBillingSet[NegBillingSet.groupby('Patient_ID').Patient_ID.transform('count') > 1]
    NegBillingSet = NegBillingSet[NegBillingSet.groupby('Patient_ID').Patient_ID.transform('count') < 51]

    
    # Randomly sample negative class for 1:1 positive negative ratio
    NegIDs = NegBillingSet['Patient_ID'].unique().tolist()
    NegSample = sample(NegIDs,len(posDF['Patient_ID'].unique()))
    NegBillingSample = NegBillingSet.loc[NegBillingSet['Patient_ID'].isin(NegSample)]
    
    return NegBillingSample

In [ ]:
# Gage patient numbers (can delete)
temp = []

In [ ]:
# Return all billing data from disease period (can be improved)
def subsetVisits(compCodes) :
    
    # Subset Billing for complication codes
    compDF = DM2DF.loc[DM2DF['DiagnosisCode_calc'].isin(compCodes)]
    
    # find relevant patientID's and compOnsets
    diseasePeriod = DiseasePeriod(compDF)
    
    # Subset Billing by patient's with complication
    bSs = DM2DF.loc[DM2DF['Patient_ID'].isin(diseasePeriod['Patient_ID'])]
    
    # Merge
    bSs = pd.merge(bSs, diseasePeriod, on='Patient_ID')
    
    # Get rid of visits outside of disease period
    relVisits = bSs[(bSs['ServiceDate'] < bSs['CompOnset']) & (bSs['ServiceDate'] >= bSs['DateOfOnset'])]
    
    #Remove patients with only 1 visit and more than 50
    relVisits = relVisits[relVisits.groupby('Patient_ID').Patient_ID.transform('count') > 1] # Change for accuracy
    relVisits = relVisits[relVisits.groupby('Patient_ID').Patient_ID.transform('count') < 51]
    temp.append(len(relVisits['Patient_ID'].unique()))
    
    #Merge with negative sample
    negSample = negativeSamples(relVisits)
    temp.append(len(negSample['Patient_ID'].unique()))
    relVisits = relVisits.append(negSample, ignore_index=True)
    
    return relVisits

In [ ]:
# Create a dataframe wih paitnet ID and visits as index, columns as diags
def patientHistories(dataframe) :
    AllVisits = {}
    for index, row in dataframe.iterrows() :
        temp = (str(row['Patient_ID']),str(row['ServiceDate']))
        diag = row['DiagnosisCode_calc']
        if temp not in AllVisits.keys():
            AllVisits[temp] = []
        AllVisits[temp] = AllVisits[temp] + [diag]
        
    PVDF = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in AllVisits.items() ]))
    PVDF = PVDF.transpose()
    PVDF = PVDF.dropna(axis=0,how='all') # Remove NA
    return PVDF

In [ ]:
# Call functions on our data
subVisits = {} # Billing data
seqVisits = {} # Multiindex DF of diags
compIDdict = {} # Patient ID's 
for k, v in complications.items() :
    compVisits = subsetVisits(v)
    subVisits[k] = compVisits
    
    CompPatients = compVisits.dropna(subset=['CompOnset'])
    CompPatients = CompPatients['Patient_ID'].unique().tolist()
    compIDdict[k] = CompPatients
    
    seqVisits[k] = patientHistories(compVisits)

### Feature vector creation

In [ ]:
# Check # of unique ICD9 codes
print(len(DM2DF['DiagnosisCode_calc'].value_counts().index))

# Remove ICD9 codes that only appear 5 or less times
ICD9s = DM2DF['DiagnosisCode_calc'].value_counts()
ICD9s = ICD9s[DM2DF['DiagnosisCode_calc'].value_counts() > 5]
print(ICD9s.index)

In [ ]:
# Function does 1) one-hot encode each visit 2) SVD to reduce dimensionality 3) Concatenate features and fills in tails
def createFeatures(df, codes, IDs) :
    RowFeatures = pd.DataFrame(data = 0,
                           index = df.index,
                           columns=codes.index)
    
    # if diag occured in a visit, col[diagnosisCode] = 1
    for index, row in RowFeatures.iterrows() :
        for diag in df.loc[index].dropna() :
            row[diag] = 1
            
    # SVD
    U, s, V = np.linalg.svd(RowFeatures)
    S = np.zeros((RowFeatures.shape[0], RowFeatures.shape[1]))
    S[:RowFeatures.shape[1], :RowFeatures.shape[1]] = np.diag(s)
    n_component = 50
    S = S[:, :n_component]
    reducedMat = U.dot(S)
    
    reducedDF = pd.DataFrame(data=reducedMat, index=RowFeatures.index)
    
    # Concatanate patient features
    # number at 1st index represents disease status
    VisitHistorySVD = {}
    for index, row in reducedDF.iterrows() :
        temp = int(index[0])
        featureVector = list(row)
        if temp not in VisitHistorySVD.keys():
            if temp in IDs :
                VisitHistorySVD[temp] = [1]
            else :
                VisitHistorySVD[temp] = [0]
        VisitHistorySVD[temp] = VisitHistorySVD[temp] + featureVector


    # Fill in to 2500
    for k,v in VisitHistorySVD.items() :
        fillLen = 2501 - len(v)
        VisitHistorySVD[k] = v + [0]*fillLen
        
    return VisitHistorySVD # Dictionary with PatientIDs as keys and encoded data as values
    

In [ ]:
# Run features (Takes a long time bc of SVD, so run over night)
features = {}
for k,v in seqVisits.items() :
    if k != 'DD' :
        print(k)
        features[k] = createFeatures(v, ICD9s, compIDdict[k])


In [ ]:
# Convert data to X and y for modeling
X, y = [], []
for k,v in VisitHistorySVD.items() :
    y.append(v[0])
    X.append(v[1:])

In [ ]:
print(y.count(1),y.count(0))

### Machine learning models

In [ ]:
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Embedding, LSTM, GlobalMaxPooling1D, SpatialDropout1D, GRU, Bidirectional, Input, TimeDistributed, Reshape

In [ ]:
# Split data
X_train, X_test, y_train, y_test = \
    train_test_split(X, y, test_size = 0.2, random_state = 42)
print(len(X_train), 'training')
print(len(X_test), 'testing')

In [ ]:
# Bidirectional GRU

# Random search drop out rate and number of units/neurons
X_train = np.array(X_train)
X_test = np.array(X_test)
input_shape = (X_train.shape[1], 1) #2500,1

units = 128


model = Sequential()
model.add(Reshape((50, 50), input_shape=(2500,)))
# 1 sees the past 
# 2 sees the past and the future


dropout = 0.2 # for regularization
print('Number of hidden units: ', units, 'Dropout: ', dropout)
model.add(Bidirectional(GRU(units, input_shape=input_shape))) # 64, $128$, 256, 512
model.add(Dropout(dropout))
model.add(Dense(1, activation='sigmoid'))

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)
print('Training...')
history = model.fit(
    np.array(X_train),
    np.array(y_train),
    batch_size = 128, 
    epochs = 15,
    validation_split = 0.1
)

print(model.evaluate(X_test, np.array(y_test)), "\n\n")

In [ ]:
# Examples
print(model.predict(X_test[0].reshape(1,2500)), y_test[0])
print(model.predict(X_test[1].reshape(1,2500)), y_test[1])

In [ ]:
# Graphing
from matplotlib import pyplot as plt

In [ ]:
# Loss per epoch
plt.clf()
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, 'g', label='Training loss')
plt.plot(epochs, val_loss, 'y', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
# Accuracy per epoch
plt.clf()
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
epochs = range(1, len(acc) + 1) #?
plt.plot(epochs, acc, 'g', label='Training acc')
plt.plot(epochs, val_acc, 'y', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
# Test with RF and MLP
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import datasets 
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from time import time

h = .02  # step size in the mesh

names = ["Random Forest", "MLP"]

classifiers = [
    RandomForestClassifier(max_depth=10, n_estimators=100),
    MLPClassifier()
]

print("{0:20}{1:40}\n-----------------------------------------------------".\
      format("Classifier", "Accuracy"))

# iterate over classifiers
for name, clf in zip(names, classifiers):
    
    start_time = time()
    clf.fit(X_train, y_train)
    score = clf.score(X_test, y_test)
    end_time = time()
    print("{0:20}{1:40}{2:40}".format(name, str(score), (end_time - start_time)))